In [95]:
import keras
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K
from keras import optimizers
import tensorflow as tf

In [96]:
def recall_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def precision_m(y_true, y_pred):
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def f1_m(y_true, y_pred):
    precision = precision_m(y_true, y_pred)
    recall = recall_m(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [154]:
img_width, img_height = 150, 150

train_data_dir = 'dataset/TDdataset/train/'
validation_data_dir = 'dataset/TDdataset/val/'
test_data_dir = 'dataset/TDdataset/test/'
nb_train_samples = 2000
nb_validation_samples = 800
epochs = 100
batch_size = 16
initial_learning_rate = 0.1


if K.image_data_format() == 'channels_first':
    input_shape = (3, img_width, img_height)
else:
    input_shape = (img_width, img_height, 3)

In [162]:
# lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
# )

opt = keras.optimizers.Adam(learning_rate=0.0001)

callback = keras.callbacks.EarlyStopping(monitor='loss', patience=5 ,mode='auto', verbose=1, min_delta=0.0001)

# callback = tf.keras.callbacks.ReduceLROnPlateau(
#     monitor='loss',
#     factor=0.1,
#     patience=10,
#     verbose=0,
#     mode='auto',
#     min_delta=0.0001,
#     cooldown=0,
#     min_lr=0.0000001,
# )

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer=opt,
              metrics=['accuracy', f1_m,precision_m, recall_m])

In [163]:
# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    shear_range=0.2,
    zoom_range=0.2,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1. / 255)

train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode='binary')

model.fit(
    train_generator,
    steps_per_epoch=nb_train_samples // batch_size,
    epochs=epochs,
    callbacks=[callback],
    validation_data=validation_generator,
    validation_steps=nb_validation_samples // batch_size)

model.save_weights('first_try.h5')

Found 2688 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Found 160 images belonging to 2 classes.
Epoch 1/100
125/125 [==============================] - 21s 158ms/step - loss: 0.6873 - accuracy: 0.5580 - f1_m: 0.5021 - precision_m: 0.5753 - recall_m: 0.5278 - val_loss: 0.6521 - val_accuracy: 0.5688 - val_f1_m: 0.4631 - val_precision_m: 0.5938 - val_recall_m: 0.3933
Epoch 2/100
125/125 [==============================] - 20s 158ms/step - loss: 0.6533 - accuracy: 0.5915 - f1_m: 0.4910 - precision_m: 0.6741 - recall_m: 0.4222
Epoch 3/100
125/125 [==============================] - 19s 148ms/step - loss: 0.6287 - accuracy: 0.6255 - f1_m: 0.4740 - precision_m: 0.7012 - recall_m: 0.3879
Epoch 4/100
125/125 [==============================] - 18s 147ms/step - loss: 0.6070 - accuracy: 0.6430 - f1_m: 0.4879 - precision_m: 0.7642 - recall_m: 0.3886
Epoch 5/100
125/125 [==============================] - 19s 151ms/step - loss: 0.5998 - accuracy: 0.6275 - f1_m: 0.4721 - preci

In [166]:
loss, accuracy, f1_score, precision, recall = model.evaluate(test_generator)

10/10 [==============================] - 0s 37ms/step - loss: 1.4079 - accuracy: 0.4750 - f1_m: 0.5015 - precision_m: 0.4664 - recall_m: 0.5680
